## Fine Tune a GPT

Companion notebook to [The Art of Hyperparameter Tuning a GPT](https://medium.com/@joel.loh.wei.en/the-art-of-hyperparameter-tuning-a-gpt-finding-harmony-between-human-machine-evaluation-part-1-of-61af780b80c4) on medium

In [1]:
# Import the "tinyshakespeare" dataset, which contains all the works of Shakespeare concatenated together 
!curl -O https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  1720k      0 --:--:-- --:--:-- --:--:-- 1718k


In [2]:
# Importing required packages
import torch
import torch.nn as nn
from torch.nn import functional as F

### "Best" base model (Model 1)

Here we have built a Transformer-based neural network that learns and predicts text by examining interactions between individual characters.
- (e.g. like 'A', 'a', '!', '?', ' ' etc).

After some tuning, we honed in on the following key parameters
- learning_rate = 4.5e-3
- batch_size = 24
- n_layer = 5
- n_head = 4
- n_embd = 128
- dropout = 0.0

In [3]:
"""
The Prologue: Setting the Stage (Hyperparameters)
"""
# Decide the structure and dynamics of the model with these hyperparameters
## Model Architecture
n_embd = 128  # The depth of understanding the model can achieve
n_head = 4  # Balancing breadth and depth of focus
n_layer = 5  # Layers of abstraction
## Training Dynamics
learning_rate = 4.5e-3  # Speed at which the model "learns"
batch_size = 24  # Number of samples to process in one iteration
block_size = 32  # Sequence length to process in each iteration; Total tokens = batch_size * block_size
dropout = 0.0  # Going wabi-sabi, embracing beauty in imperfection
## Training control
max_iters = 5000  # Hard stop number for training iterations
eval_interval = 100  # How often to evaluate model on training set
eval_iters = 200  # Number of "text chunks" to sample 
## Computational setup
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # For all you NVDIA cool kids
## Early stopping setup to terminate training if validation performance doesn't improve
patience = 10  # Number of iterations to wait without improvement
best_val_loss = float('inf')  # Track the best validation loss

"""
Prelude to Creation: Pre-Processing Rituals
"""
# Set the random seed for reproducibility
torch.manual_seed(1337)

# Pre-process data
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Translating raw text to a language our model understands using characters
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train-test split: 90% training and 10% validation
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) 
train_data = data[:n]
val_data = data[n:]

"""
Setting the Scene: Under the hood of our GPT
"""
# Function to generate mini-batches for training and validation
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# creating a simple transformer
class SimpleGPT(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = SimpleGPT()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

"""
Plot twist! Do you even "Iterative Tango" bro?
"""
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        
        # Check for early stopping
        if losses['val'] < best_val_loss:
            best_val_loss = losses['val']
            interval_without_improvement = 0  # reset the count
        else:
            interval_without_improvement += 1

        if interval_without_improvement >= patience:
            print("Early stopping due to no improvement in validation loss.")
            break

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

"""
The Grand Finale: Text Generation
"""
# Time for some faux-Shakespeare
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


1.010497 M parameters


/Users/joelloh/Desktop/Data Science Practice/BuildGPTLecture/lect_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


step 0: train loss 4.3974, val loss 4.3978
step 100: train loss 2.3922, val loss 2.3845
step 200: train loss 2.1830, val loss 2.2172
step 300: train loss 2.0972, val loss 2.1472
step 400: train loss 2.0086, val loss 2.0895
step 500: train loss 1.9509, val loss 2.0498
step 600: train loss 1.8827, val loss 2.0157
step 700: train loss 1.8745, val loss 1.9972
step 800: train loss 1.8370, val loss 1.9649
step 900: train loss 1.8155, val loss 1.9481
step 1000: train loss 1.7768, val loss 1.9290
step 1100: train loss 1.7587, val loss 1.9102
step 1200: train loss 1.7458, val loss 1.9045
step 1300: train loss 1.7245, val loss 1.8765
step 1400: train loss 1.7051, val loss 1.8649
step 1500: train loss 1.6965, val loss 1.8665
step 1600: train loss 1.6830, val loss 1.8362
step 1700: train loss 1.6815, val loss 1.8320
step 1800: train loss 1.6704, val loss 1.8261
step 1900: train loss 1.6564, val loss 1.8228
step 2000: train loss 1.6495, val loss 1.8283
step 2100: train loss 1.6531, val loss 1.8287
